In [1]:
import numpy as np
import cv2
import glob
import json
import dlib
from imutils import face_utils
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras_vggface.vggface import VGGFace
import face_recognition
import math

Using TensorFlow backend.


In [2]:
participants = glob.glob('dataset/*')
# participants = sorted(participants)

In [8]:
# Load the dataset
with open('data_cleaned.json') as json_file:
    data_all = json.load(json_file)
# Extract the keys in sorted order
keys_all = sorted(data_all)
# Convert python list to np array
keys_all = np.asarray(keys_all)

with open('uuid_lengths_128.json') as json_file:
    uuid_lengths_128 = json.load(json_file)

In [4]:
DATASET_SIZE = len(keys_all)

In [8]:
# fail_counter = 0
# for i in range(50):
#     key = keys_all[i]
    
#     im = cv2.imread('dataset/' + key)
#     im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
#     label = key.split('/')[1]
    
#     # detect the (x, y)-coordinates of the bounding boxes
#     # corresponding to each face in the input image
#     boxes = face_recognition.face_locations(im, model='hog')
#     if len(boxes) == 0:
#         print('Failed to detect faces')
#         fail_counter += 1
#         continue
        
#     # Find the correct box assuming that it is the biggest one
#     max_dist = 0
#     for candidate_box in boxes:
#         diagonal_dist = math.sqrt(((candidate_box[0] - candidate_box[2])**2 + (candidate_box[1] - candidate_box[3])**2))
#         if diagonal_dist >= max_dist:
#             max_dist = diagonal_dist
#             box = candidate_box
#         else:
#             continue
    
#     # compute the facial embedding for the face
#     encodings = face_recognition.face_encodings(im, [box])
#     encodings = encodings[0]

In [15]:
# Loop over each participant
#uuid_lengths_128 = {}
fail_counter = 0
for j in range(len(participants)):
    
    # Extract the UUID
    uuid_excluded = participants[j].split('/')[1]

    # Loop over the dataset to remove the examples associated with this participant
    indices_excluded = []
    keys_excluded = []
    for i in range(DATASET_SIZE):
        key = keys_all[i]
        uuid = key.split('/')[0]
        if(uuid == uuid_excluded):
            indices_excluded.append(i)
            keys_excluded.append(key)
    
    # Construct the validation dataset consisting of examples of a single participant
    # Evaluation of the classifiers will be done on this dataset
    encodings = np.zeros((len(keys_excluded), 128))
    labels = np.zeros(len(keys_excluded))
    
    failed_indices = []
    for i in range(len(keys_excluded)):
        key = keys_excluded[i]

        im = cv2.imread('dataset/' + key)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        label = key.split('/')[1]

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input image
        boxes = face_recognition.face_locations(im, model='hog')
        if len(boxes) == 0:
            print('Failed to detect faces for uuid: ' + uuid_excluded)
            failed_indices.append(i)
            fail_counter += 1
            continue

        # Find the correct box assuming that it is the biggest one
        max_dist = 0
        for candidate_box in boxes:
            diagonal_dist = math.sqrt(((candidate_box[0] - candidate_box[2])**2 + (candidate_box[1] - candidate_box[3])**2))
            if diagonal_dist >= max_dist:
                max_dist = diagonal_dist
                box = candidate_box

        # compute the facial embedding for the face
        encoding = face_recognition.face_encodings(im, [box])[0]
        encodings[i, :] = encoding
        if(label=='positive'):
            labels[i] = 1
        else:
            labels[i] = 0
        if(uuid_excluded == '0656f5fe35a54d1589e526a702f578b0'):
            print(labels[i])
            
    
    encodings = np.delete(encodings, failed_indices, axis=0)
    labels = np.delete(labels, failed_indices, axis=0)
    uuid_lengths_128[uuid_excluded] = labels.shape[0]
    np.save('encodings_128/encodings_' + uuid_excluded , encodings)
    np.save('encodings_128/labels_' + uuid_excluded, labels)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to detect faces for uuid: f4e9d804730947a78e8251b35f2f519e
Failed to 

In [16]:
# with open('uuid_lengths_128.json', 'w') as fp:
#     json.dump(uuid_lengths_128, fp)